# Ahvenanmaa Geta infinity lineaments

In [ ]:
import os, sys
sys.path.append("..")

In [2]:
from pathlib import Path
import pathlib
import geopandas as gpd
from tval import trace_validator
import time

/home/nialov/.local/share/virtualenvs/trace-validator-GOA3psTM/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


In [3]:
# Setup snap thresholds here
snap_threshold = 0.001
snap_threshold_multiplier = 1.1
area_edge_snap_multiplier = 5

In [4]:
import numpy as np

In [5]:
gdf = gpd.read_file("/mnt/f/Users/nikke/Documents/projects/Academic_projects/gtk_ahvenanmaa/Ahvenanmaa_LiDAR/Ahvenanmaa_LiDAR_traces/Geta_LiDAR_lineaments_infinity_scale_traces_validated.gpkg")

In [6]:
gdf.head()

,OBJECTID,Lineament_ID,Lineament_Name,Data_Source,Scale,Certainty,Geological_Type,Observation,Remarks,Operator,Date,Structure_Name,Fault_System,SHAPE_Length,VALIDATION_ERRORS,geometry
0,251.0,LiDAR_A246,None,LiDAR,1:500 000/1:200 000,1_Geoph_OR_LiDAR,None,None,None,Kraatz S.,2020-06-17T00:00:00,None,None,37266.895093,[],"LINESTRING (91580.109 6717268.093, 100505.404 ..."
1,256.0,LiDAR_A251,None,LiDAR,1:500 000/1:200 000,1_Geoph_OR_LiDAR,None,None,None,Kraatz S.,2020-06-18T00:00:00,None,None,24925.409207,[],"LINESTRING (88164.323 6704140.141, 91537.099 6..."
2,261.0,LiDAR_C255,None,LiDAR,1:500 000/1:200 000,1_Geoph_OR_LiDAR,None,None,A->C,Kraatz S.,2020-06-18T00:00:00,None,None,24121.249821,[],"LINESTRING (101401.696 6695046.745, 101712.509..."
3,870.0,LiDAR_B1505,South-Finland_Shearzone,LiDAR,1:200 000,1_Geoph_OR_LiDAR,Shear_zone,None,None,Kraatz S.,2020-06-17T00:00:00,None,None,61211.451796,[],"LINESTRING (98815.452 6727588.839, 107626.094 ..."
4,878.0,LiDAR_C1512,None,LiDAR,1:200 000,1_Geoph_OR_LiDAR,None,None,None,Kraatz S.,2020-06-17T00:00:00,None,None,13749.234433,[],"LINESTRING (100374.387 6713481.752, 100381.964..."


In [7]:
lin_id = 'Lineament_ID'
date = 'Date'
operator = 'Operator'
certainty = 'Certainty'
scale = 'Scale'
geol_type = 'Geological_Type'
data_source = 'Data_Source'
remarks = "Remarks"

In [8]:
cols = (lin_id,
date,
operator,
certainty,
scale,
geol_type,
data_source,
remarks,)


In [9]:
assert all([col in gdf.columns for col in cols])

In [10]:
put_date = '2020-09-22T00:00:00'
put_scale = 'Infinity'
put_scale_200k = '1:200 000/Infinity'
put_scale_500k = '1:500 000/1:200 000/Infinity'
put_certainty = '1_Geoph_OR_LiDAR'
put_data_source = 'LiDAR'
put_operator = "Ovaskainen N."

In [11]:
def remarks_det(linid, remarks_str):
    if remarks_str is None:
        remarks_str = ""
    if not isinstance(remarks_str, str):
        raise TypeError(f"remarks_str not str: {remarks_str}")
    if linid is not None:
        assert isinstance(linid, str)
        if "_B" in linid:
            return f"B->C {remarks_str}"
        elif "_A" in linid:
            return f"A->C {remarks_str}"
        elif len(remarks_str) == 0:
            return f"B->C {remarks_str}"
        else:
            return remarks_str
    else:
        return remarks_str
    
def scale_det(scale_str):
    if scale_str is not None:
        assert isinstance(scale_str, str)
        if "500" in scale_str:
            return put_scale_500k
        elif "200" in scale_str:
            return put_scale_200k
        else:
            raise ValueError(f"Unknown scale: {scale_str}")
    else:
        return put_scale

def linid_det(linid, num):
    if linid is not None:
        assert isinstance(linid, str)
        if "_B" in linid:
            return linid.replace("_B", "_C")
        if "_A" in linid:
            return linid.replace("_A", "_C")
        else:
            return linid
    else:
        next_num = next(num)
        assert isinstance(next_num, int)
        assert next_num > 29999
        return f"LiDAR_D{next_num}"

In [12]:
def genera(start, end):
    for i in range(start, end):
        yield i
gene = genera(30000, 50000)

In [13]:
new_gdf = gdf.copy()

In [14]:
new_gdf[remarks] = [remarks_det(linid, r) for linid, r in zip(new_gdf[lin_id], new_gdf[remarks])]
new_gdf[lin_id] = [linid_det(linid, gene) for num, linid in enumerate(new_gdf[lin_id])]
new_gdf[date] = [put_date if val is None else val for val in new_gdf[date]]
new_gdf[operator] = [put_operator for val in new_gdf[operator]]
new_gdf[certainty] = [put_certainty if val is None else val for val in new_gdf[certainty]]
new_gdf[data_source] = [put_data_source if val is None else val for val in new_gdf[data_source]]
new_gdf[scale] = [scale_det(s) for s in new_gdf[scale]]

In [18]:
new_gdf.crs = "EPSG:3067"

In [19]:
new_gdf.to_file("/mnt/f/Users/nikke/Documents/projects/Academic_projects/gtk_ahvenanmaa/Ahvenanmaa_LiDAR/Ahvenanmaa_LiDAR_traces/Geta_LiDAR_lineaments_infinity_scale_traces_final.gpkg", driver="GPKG")

In [17]:
["B" in lid for lid in gdf[lin_id] if lid is not None]

[False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True]